In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
#%matplotlib notebook
plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
df = pd.read_csv("Advertising.csv")
df.head()

,TV,radio,newspaper,sales
0,230.100,37.800,69.200,22.100
1,44.500,39.300,45.100,10.400
2,17.200,45.900,69.300,9.300
3,151.500,41.300,58.500,18.500
4,180.800,10.800,58.400,12.900


In [3]:
from sklearn.model_selection import train_test_split

X = df.drop("sales", axis=1)
y = df["sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 101)

In [4]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def train_val(model, X_train, y_train, X_test, y_test):
    
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    scores = {"train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    "test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [5]:
from sklearn.neural_network import MLPRegressor

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

operations = [("scaler", StandardScaler()), ("mlp", MLPRegressor())]
Pipeline(steps=operations)
model = Pipeline(steps=operations)
model.fit(X_train, y_train)

train_val(model, X_train, y_train, X_test, y_test)

,train,test
R2,-1.264,-0.867
mae,7.072,6.786
mse,59.840,55.290
rmse,7.736,7.436


In [7]:
from sklearn.model_selection import cross_validate

operations = [("scaler", StandardScaler()), ("mlp", MLPRegressor())]
model = Pipeline(operations)

scores = cross_validate(model, X_train, y_train, scoring=['r2', 
            'neg_mean_absolute_error','neg_mean_squared_error','neg_root_mean_squared_error'], cv =10)
df_scores = pd.DataFrame(scores, index = range(1, 11))
df_scores.mean()[2:]

test_r2                             -1.410
test_neg_mean_absolute_error        -6.839
test_neg_mean_squared_error        -54.935
test_neg_root_mean_squared_error    -7.387
dtype: float64

In [11]:
from sklearn.model_selection import GridSearchCV

operations = [("scaler", StandardScaler()), ("mlp", MLPRegressor())]
Pipeline(steps=operations)
model = Pipeline(steps=operations)

model.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()), ('mlp', MLPRegressor())],
 'verbose': False,
 'scaler': StandardScaler(),
 'mlp': MLPRegressor(),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'mlp__activation': 'relu',
 'mlp__alpha': 0.0001,
 'mlp__batch_size': 'auto',
 'mlp__beta_1': 0.9,
 'mlp__beta_2': 0.999,
 'mlp__early_stopping': False,
 'mlp__epsilon': 1e-08,
 'mlp__hidden_layer_sizes': (100,),
 'mlp__learning_rate': 'constant',
 'mlp__learning_rate_init': 0.001,
 'mlp__max_fun': 15000,
 'mlp__max_iter': 200,
 'mlp__momentum': 0.9,
 'mlp__n_iter_no_change': 10,
 'mlp__nesterovs_momentum': True,
 'mlp__power_t': 0.5,
 'mlp__random_state': None,
 'mlp__shuffle': True,
 'mlp__solver': 'adam',
 'mlp__tol': 0.0001,
 'mlp__validation_fraction': 0.1,
 'mlp__verbose': False,
 'mlp__warm_start': False}

In [12]:
param_grid = {'mlp__alpha': [0.01,0.1,0.02, 0.005],
              'mlp__hidden_layer_sizes': [(20,20), (100,50,150),(300,200,150)],
              'mlp__activation': ['relu', 'logistic']}

mlp_model_grid = GridSearchCV(model, param_grid, scoring="neg_mean_squared_error", cv=10)
mlp_model_grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('mlp', MLPRegressor())]),
             param_grid={'mlp__activation': ['relu', 'logistic'],
                         'mlp__alpha': [0.01, 0.1, 0.02, 0.005],
                         'mlp__hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                     (300, 200, 150)]},
             scoring='neg_mean_squared_error')

In [13]:
mlp_model_grid.best_params_

{'mlp__activation': 'relu',
 'mlp__alpha': 0.1,
 'mlp__hidden_layer_sizes': (300, 200, 150)}

In [14]:
train_val(mlp_model_grid, X_train, y_train, X_test, y_test)

,train,test
R2,0.990,0.986
mae,0.330,0.441
mse,0.255,0.406
rmse,0.505,0.637
